Author: Maxime Marin \
@: mff.marin@gmail.com
\
\
<font color='blue' size = 8 > <strong> 1) INTRODUCTION </strong>  </font>\
\
<font size = 4> The next few notebooks aim to provide case studies using the cloud cluster availability demonstrated earlier, in order to investigate, load, visualise analyse and extract data stored on external servers.\
Main advantages include speed, ease of use and repeatability. Although we are using a python enviornment, novice python users can following the receipe and adapt them to their needs.
\
\
Sections will be divided into different notebooks focusing on different tasks including:</font>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font size = 4><font color='green'><strong>  - Notebook 1 Start:</strong> </font> Imports libraries, fires up the cluster, provides quick description of datasets available and loads dataset of interest.\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- Notebook 2 Interactive:</strong></font> Provides interactive tools to investigate a the chosen dataset and make some quick plots.\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- Notebook 3 Analysis:</strong></font> Performs further analysis on the chosen dataset including climatology, linear trends and anomalies.\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- Notebook 4 Data Extraction:</strong></font> Extracts data into a format of choice and saves it locally for the user to perform more in-depth analysis .</font>\



***
<font color='blue' size =8> <strong> 2) LIBRARIES </strong>  </font>\
\
<font size = 4>At the beginning of all notebooks, it is necessary to load the packages that will be used to run the code. Note that some libraries might be called within other "bits" of code hidden in other files.\
"importing" a librarie or package supposes that it was loaded in the python environment  prior to running the notebooks... Thankfully we have taken care of that. </font>


In [1]:
import sys
#Ensure latest version of fsspec is used to allow simpletemplate=True of reference file system
sys.path.insert(0,'/home/jovyan/filesystem_spec')

import xarray as xr 
import numpy as np
import pandas as pd
import cmocean
from scipy import stats
import matplotlib.pyplot as plt
import cmocean
import cartopy
import cartopy.crs as ccrs
import datetime
import fsspec
import xarray as xr
import os, glob
from time import strptime
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%run SST_fun.ipynb

***
<font color='blue' size = 8 > <strong> 3) CLUSTER FIRING </strong>  </font>\
\
<font  size = 4 >Let's now create a cluster to allow for an exponentional increase in speed.\
For comparison, typical laptops have just 8-16 Gb of memory and run on 4-8 CPUs...</font>\

In [2]:
## change after Paul's commit
gateway_cluster = True
from dask.distributed import Client, LocalCluster
from dask_gateway import Gateway

# Dask gateway
if gateway_cluster:
    gateway = Gateway()
    clusters = gateway.list_clusters()
    if not clusters:
        print('Creating new cluster. Please wait for this to finish.')
        options = gateway.cluster_options()
        options.worker_cores = 1
        options.worker_memory = 1.0
        cluster = gateway.new_cluster(cluster_options=options)
    else:
        print(f'An existing cluster was found. Connected to cluster \033[1m{clusters[0].name}\033[0m')
        cluster=gateway.connect(clusters[0].name)
        cluster.adapt(minimum=1, maximum=30)
    #cluster.scale(180)
    client = cluster.get_client()
    #client.wait_for_workers(n_workers=180)

# Local cluster
else:
    cluster = LocalCluster()
    client = Client(cluster)

display(cluster)
display(client)

Creating new cluster. Please wait for this to finish.


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.csiro.easi-eo.solutions/services/dask-gateway/clusters/easihub.0bf1175c95d2493b9ce09ece08d9fc17/status,


<font  size = 4 >Let's click on the "Dashboard" link to access all details of the cluster we have created.</font>

***
<font color='blue' size = 8 > <strong> 4) DATA DESCRIPTION </strong>  </font>

<font size = 4> Finally, let's have a look at what the available datasets are, along with some information about their metadata.\
\
For this workshop, we have provided users with a choice between:</font>
  
<font size = 4>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- IMOS SST:</strong></font> Satellite SST product created by IMOS hosted on the AODN servers.\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- BRAN 2020:</strong></font> The CSIRO BlueLink Reanalysis product hosted on NCI. Only a few variables have been chosen for the sake of the workshop\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- IMOS Chlorophyll:</strong></font> Satellite Chlorophyll product created by IMOS hosted on the AODN servers..\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'><strong>- IMOS SLA:</strong></font> Sea Level Anomaly derived from satellite, created by IMOS and hosted on the AODN servers.</font>

In [2]:
# Interactive panel to list details of different available products, including variables, coordinates, time step and text description.
#@interact
#def show_timeseries(coord = ['lon','lat','time']):
#    return data[coord]
  

In [144]:
w1 = widgets.Dropdown(options = ["Single","Area"],description='Spatial')
display(w1)

if w1 == 'Single':
    w2 = widgets.FloatText(description='Lon')
    w3 = widgets.FloatText(description='Lat')
else

    
display(w2)

Dropdown(description='Spatial', options=('Single', 'Area'), value='Single')

FloatText(value=0.0, description='Lon')

In [3]:
#cluster.shutdown()